# Project 2: fire in the nature park

Laura Santa Cruz<br>
Intelligent data Analysis and Machine learning <br>
University of Potsdam 2024<br>

## 1. Introduction and problem setting


## 2. Data preparation and normalization

## 3. Model creation

## 4. Summary

## 5. Conclusion